## 2개의 층
케라스 api로 패션 mnist 데이터셋을 불러오겠습니다.


In [1]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


픽셀값 범위를 0~1사이, 28x28 크기의 2차원 배열을 784 크기 1차원 배열로 펼치고 사이킷런의 train_test_split() 함수로 훈련 세트와 검증 세트로 나눈다.

In [2]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size = 0.2, random_state = 42
)

이제 인공 신경망 모델에 층을 2개 추가해보자.

입력층과 출력층 사이에 밀집층이 추가된 것이다. 이 사이의 모든 층을 은닉 층이라고 부른다.

은닉층에는 주황색 원으로 활성화 함수가 표시되어 있다. 활성화 함수는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수. 소프트맥스 함수도 활성화 함수이다.

출력층에 적용하는 활성화 함수는 종류가 제한되어 있다.

이진 분류일 경우 시그모이드, 다중 분류일 경우 소프트맥스 함수 사용.

은닉층에 왜 활성화 함수를 적용할까? 다음 2개의 선형 방정식을 생각해보자.

    a * 4 + 2 = b
                    ->a * 12 + 1 = c                
    b * 3 - 5 = c

왼쪽의 첫 번째 식에서 계산된 b가 두 번째 식에서 c를 계산하기 위해 쓰인다. 하지만 두 번째 식에 첫번째 식을 대입하면 하나로 합쳐진다. 이렇게 되면 b는 사라지면서 b가 하는 일이 없어지는 셈이다.

신경망도 마찬가지로 은닉층에서 선형적인 산술 계산만 수행하면 수행 역할이 없는 셈이다. 선형 계산을 적당하게 비 선형적으로 비틀어 주어야 다음 층의 계산과 단순히 합쳐지지 않고 나름의 역할을 할 수 있다.

시그모이드 함수는 뉴런의 출력 z값을 0과 1 사이로 압축한다. 이 시그모이드 활성화 함수를 사용한 은닉층과 소프트 맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들어 보자.



In [3]:
dense1 = keras.layers.Dense(100, activation='sigmoid',input_shape=(784,))
dense2 = keras.layers.Dense(10, activation = 'softmax')

dense1이 은닉층이고 100개의 뉴런을 가진 밀집층. 활성화 함수를 sigmoid로 지정, input_shape 매개변수에서 입력의 크기를 지정.
여기서 은닉층의 뉴런 개수를 정하는 데는 기준이 없다.

여기서 한가지 제약사항은 적어도 출력층의 뉴런보다는 많게 만들어야 한다는 것이다. 클래스 10개에 대한 확률을 예측해야 하는데 이전 은닉층의 뉴런이 10개보다 적으면 부족한 정보가 전달된다.

dense2는 출력층. 10개의 클래스를 분류하므로 10개의 뉴런을 두고 활성화 함수는 소프트 맥스 함수로 지정.

## 심층 신경망 만들기
dense1, dense2 객체를 Sequential 클래스에 추사하여 심층 신경망을 만들어 보자.

In [4]:
model = keras.Sequential([dense1, dense2])

Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 층 객체를 리스트로 전달한다. 여기서 출력층을 가장 마지막에 두어야 한다.

케라스는 모델의 summary() 메서드를 호출하면 층에 대한 정보를 얻을 수 있다.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


첫줄에는 모델 이름, 은닉층, 출력층 순서대로 나옴

층을 만들 때 name 매개변수로 이름을 지정할 수 있다.

출력 크기는 (None, 100)이다. None는 샘플의 개수를 나타낸다. 샘플의 개수가 정해지지 않아서이다. 케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한번에 사용하지 않고 잘게 나누어 여러번에 걸쳐 경사 하강법 단계를 수행한다.

케라스의 기본 미니 배치 크기는 32개이다. 이 값은 fit() 메서드에서 batch_size 매개변수로 바꿀 수 있다. 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응하도록 None으로 설정.

두번째 100은 뉴런의 개수

마지막으로 모델의 파라미터 개수가 출력. 이 층은 Dense 층이므로 입력 픽셀의 784개와 100개의 모든 조합에 대한 가중치. 그리고 뉴런마다 1개의 절편이 있다.

summary() 메서드의 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 나온다. 은닉층과 출력층의 파라미터 개수를 합친 것이다. 그 아래는 훈련되지 않는 파라미터는 0으로 나온다.

## 층을 추가하는 다른 방법
처음에는 Dense클래스의 객체 1,2를 만들어 Sequential클래스에 전달했다. 이 두 객체는 저장하여 쓸 일이 없기 때문에 Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만든다.

In [6]:
model = keras.Sequential([
                          keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,),name = 'hidden'),
                          keras.layers.Dense(10, activation = 'softmax', name = 'output')
], name = '패션 MNIST 모델')

In [7]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


많은 층을 추가하려면 모델의 add() 메서드를 사용한다.

In [8]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이제 모델을 훈련해보자. compile()메서드 설정은 동일하게 한다 5번의 에포크 동안 훈련.


In [10]:
model.compile(loss='sparse_categorical_crossentropy',metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.7631 - accuracy: 0.7566
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4108 - accuracy: 0.8529
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3718 - accuracy: 0.8655
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3534 - accuracy: 0.8729
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3285 - accuracy: 0.8816


훈련 세트에 대한 성능을 보면 추가된 층이 성능을 향상시켰다.

다음에는 이미지 분류 문제에서 높은 성능을 낼 수 있는 활성화 함수에 대해 알아보자.

## 렐루 함수
렐루 함수는 간단하다. 입력이 양수일 경우 활성화 함수가 없는 것처럼 입력을 통과시키고 음수일 경우에는 0으로 만든다.

렐루 함수는 max(0,z)와 같이 쓸 수 있다. z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력한다.

패션MNIST 데이터는 28*28크기이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의 reshape() 메서드를 통해 1차원으로 펼쳤다. 케라스에서는 이를 위한 Flatten층을 제공한다.

Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다. 따라서 인공 신경망의 성능을 위해 기여하는 바는 없지만 Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 이를 층이라고 부른다. Flatten층은 입력층 바로 뒤에 추가한다.

In [11]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [13]:
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()
train_scaled = train_input/255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [14]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=10)

Epoch 1/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6775 - accuracy: 0.7667
Epoch 2/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3985 - accuracy: 0.8580
Epoch 3/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3569 - accuracy: 0.8729
Epoch 4/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3329 - accuracy: 0.8806
Epoch 5/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3189 - accuracy: 0.8861
Epoch 6/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3099 - accuracy: 0.8918
Epoch 7/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3015 - accuracy: 0.8920
Epoch 8/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2907 - accuracy: 0.8982
Epoch 9/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2753 - accuracy: 0.9022
Epoch 10/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.275

In [15]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3863 - accuracy: 0.8757


[0.3863057494163513, 0.8756666779518127]

## 옵티마이저
사람이 지정해주어야 하는 파라미터 = 하이퍼 파라미터

여러개의 은닉층을 더 추가할 수도 있다. 추가할 은닉층의 개수는 모델이 학습하는 것이 아니라 우리가 지정해야 한다.

케라스의 기본적으로 미니배치 경사 하강법을 사용하며 미니배치 개수는 32개이다. fit()메서드의 batch_size 매개변수에서 이를 조정할 수 있으며 이 역시 하이퍼 파라미터이다. 또한 fit() 메서드의 epochs 매개변수도 하이퍼 파라미터 이다. 반복 횟수에 따라 다른 모델이 만들어지기 때문이다.

마지막으로 compile() 메서드에서는 케라스의 기본 경사 하강법 알고리즘인 RMSprop을 사용한다. 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공하고 이를 옵티마이저라고 부른다.

가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD이다. 1개의 샘플을 뽑아서 훈련하지 않고 앞서 언급한 것처럼 기본적으로 미니배치를 사용한다.

In [16]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

이 옵티마이저는 tensorflow.keras.optimizers패키지 아래 SGD 클래스로 구현되어 있다. sgd 문자열은 이 클래스의 기본 설정 매개 변수로 생성한 객체와 동일하다. 즉 아래 코드와 동일하다.

In [19]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss = 'sparse_categorical_crossentropy', metrics='accuracy')

옵티마이저에 'sgd'를 지정하면 자동으로 SGD 클래스 객체를 만들어준다.
 또한 SGD클래스의 학습률(0.01)을 바꾸고 싶을 때는 다음과 같은 learning_rate 매개변수에 지정하여 사용한다.

In [18]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

SGD외에 다양한 옵티마이저들이 있다.
* 기본 경사 하강법 옵티마이저
        SGD, 모멘텀, 네스테로프 모멘텀
* 적응적 학습률 옵티마이저
        RMSprop, Adam, Adagrad

기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공. SGD클래스의 momentum 매개변수의 기본 값은 0이다. 이를 0보다 큰 값으로 지정하면 이전 그레이디언트를 가속도처럼 사용하는 모멘텀 최적화를 사용한다. 보통 momentum 매개변수는 0.9이상을 지정한다.

다음처럼 SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면 네스테로프 모멘텀 최적화(네스테로프 가속 경사)를 사용한다.



In [ ]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현. 대부분 경우 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공한다.

모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. 이렇게 하면 안정적이고 최적점에 수렴할 가능성이 높다. 이런 학습률을 적응적 학습률이라고 한다.

적응적 학습률을 대표하는 옵티마이저는 Adagrad와 RMSprop이다. 각각 compile() 메서드의 optimizer 매개변수에 'adagrad'와 'rmsprop'으로 지정할 수 있다. optimizer 매개변수의 기본값이 바로 'rmsprop'이다. 두 옵티마이저의 매개변수를 바꾸고 싶으면 SGD와 같이 Adagrad와 RMSprop 클래스 객체를 만들어 사용한다.

In [ ]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam이다. 이는 keras.optimizers 패키지 아래에 모두 있다. 적응적 학습률을 사용하는 이 3개의 클래스는 learning_rate 매개변수의 기본값으로 모두 0.001을 사용한다.

Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST모델을 훈련해보자.

In [21]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [22]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6671 - accuracy: 0.7669
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3965 - accuracy: 0.8573
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3550 - accuracy: 0.8742
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3246 - accuracy: 0.8802
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3009 - accuracy: 0.8903


In [23]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3526 - accuracy: 0.8770


[0.3525923788547516, 0.8769999742507935]